In [7]:
import numpy as np
import pandas as pd  # For data manipulation
import matplotlib.pyplot as plt  # For visualization
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import warnings



def simulate_space_launch_data(n_samples=300, random_state=42):
    """
    Simulate synthetic data for space launch vehicles using a simplified rocket equation.
    
    Features:
      - thrust (in kN): Engine thrust, uniformly distributed between 500 and 2000 kN.
      - fuel_mass (in tons): Total fuel mass, uniformly distributed between 100 and 2000 tons.
      - payload_mass (in tons): Payload mass, uniformly distributed between 1 and 100 tons.
      
    Constant:
      - dry_mass (in tons): Fixed at 50 tons (vehicle's structural mass without fuel).
      
    Physics-Based Target (max_altitude in meters):
      1. Compute the initial mass: m0 = dry_mass + fuel_mass + payload_mass.
      2. Compute the final mass after fuel burn: m_final = dry_mass + payload_mass.
      3. Adjust the nominal exhaust velocity (v_e = 3000 m/s) by an engine efficiency factor 
         derived from thrust (normalized by 1000 kN): v_e_effective = v_e * (thrust / 1000).
      4. Use the Tsiolkovsky rocket equation: Δv = v_e_effective * ln(m0 / m_final).
      5. Approximate maximum altitude (ignoring drag and gravity losses) as:
             max_altitude ≈ (Δv)^2 / (2 * g),
         where g = 9.81 m/s².
      6. Add Gaussian noise to simulate real-world measurement error.
    
    Returns:
      X: Feature matrix of shape (n_samples, 3) containing [thrust, fuel_mass, payload_mass].
      y: Target vector (max_altitude) of shape (n_samples,).
    """
    np.random.seed(random_state)
    
    # Generate features:
    thrust = np.random.uniform(500, 2000, n_samples)         # Thrust in kN
    fuel_mass = np.random.uniform(100, 2000, n_samples)        # Fuel mass in tons
    payload_mass = np.random.uniform(1, 100, n_samples)        # Payload mass in tons
    dry_mass = 50  # Constant dry mass in tons
    
    # Constants for the rocket equation:
    v_e = 3000  # Nominal effective exhaust velocity in m/s
    g = 9.81    # Gravitational acceleration in m/s²
    
    # Compute initial and final masses:
    m0 = dry_mass + fuel_mass + payload_mass   # Initial total mass
    m_final = dry_mass + payload_mass          # Final mass after fuel burn
    
    # Calculate engine efficiency factor based on thrust (normalized by 1000 kN)
    efficiency_factor = thrust / 1000  # Dimensionless factor
    
    # Compute the effective exhaust velocity:
    v_e_effective = v_e * efficiency_factor
    
    # Compute delta-v using the Tsiolkovsky rocket equation:
    delta_v = v_e_effective * np.log(m0 / m_final)
    
    # Estimate maximum altitude using an energy approximation: altitude ≈ (delta_v)^2 / (2 * g)
    max_altitude = (delta_v**2) / (2 * g)
    
    # Add Gaussian noise to simulate measurement error and real-world variability:
    noise = np.random.normal(0, 10000, n_samples)  # Standard deviation of 10,000 m
    max_altitude += noise
    
    # Combine features into a single matrix and return with the target:
    X = np.column_stack((thrust, fuel_mass, payload_mass))
    y = max_altitude
    return X, y

def evaluate_model(model, X, y, cv=5):
    """
    Evaluate a regression model using cross-validation.
    
    Parameters:
      model: Regression model instance to be evaluated.
      X: Feature matrix.
      y: Target vector.
      cv: Number of cross-validation folds (default is 5).
    
    Returns:
      The mean R² score across the specified cross-validation folds.
    """
    scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
    return scores.mean()

# --------------------------- Main Script --------------------------- #

# Generate the synthetic dataset for space launch vehicles
X, y = simulate_space_launch_data(n_samples=300)

# Define a dictionary of regression models to compare:
models = {
    'Linear Regression': LinearRegression(),  # Baseline linear model
    'Polynomial Regression (Degree 2)': make_pipeline(
        PolynomialFeatures(degree=2, include_bias=False),  # Transform features to polynomial terms (degree 2)
        LinearRegression()  # Apply linear regression on transformed features
    ),
    'Random Forest Regression': RandomForestRegressor(n_estimators=100, random_state=42)  # Ensemble model for non-linear relationships
}

# Evaluate each model using 5-fold cross-validation and print the R² score:
results = {}
for model_name, model in models.items():
    score = evaluate_model(model, X, y)
    results[model_name] = score
    print(f"{model_name}: R² Score = {score:.3f}")

# Determine the best model based on the highest R² score:
best_model_name = max(results, key=results.get)
print("\nBest Model:", best_model_name, "with R² Score =", results[best_model_name])


Linear Regression: R² Score = 0.814
Polynomial Regression (Degree 2): R² Score = 0.988
Random Forest Regression: R² Score = 0.961

Best Model: Polynomial Regression (Degree 2) with R² Score = 0.9877691179293565
